In [1]:
import numpy as np
import json
import os
import pandas as pd
import matplotlib.pyplot as plt

from utils import load_env_file, set_mpl_configs
from utils import leave_percentile, distribution_analysis

load_env_file()
set_mpl_configs()

DATA_DIR = os.getenv('DATA_DIR')
print('DATA_DIR: {}'.format(DATA_DIR))

load env file
  root dir:
    /Users/k/Repo/gp-ibd
  current system:
    Darwin
  load .env.darwin
  loaded data dir:
    /Users/k/Nutstore Files/毕设-EHR/DB
done.
set matplotlib configs
  font family:
    ['Times New Roman']
done.
DATA_DIR: /Users/k/Nutstore Files/毕设-EHR/DB


In [162]:
with open('../data/ibd_demo.json', 'r') as f:
    data = json.loads(f.read())
    
both_ibd_patients: list = data['both_ibd']
only_uc_patiens: list = data['only_uc']
only_cd_patients: list = data['only_cd']


df = pd.read_csv(os.path.join(DATA_DIR, 'Labevent','Labevents_Death.csv'))
df = df[df.subject_id != 12468016]
df.head()

/var/folders/lk/f71xxpxj0s5357hrf4g5qp400000gn/T/ipykernel_71320/4103536517.py:9: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(DATA_DIR, 'Labevent','Labevents_Death.csv'))


,subject_id,itemid,label,value,valuenum,valueuom,dod
0,10098672,51250,MCV,88,88.0,fL,NaN
1,10098672,51200,Eosinophils,1.5,1.5,%,NaN
2,10098672,51250,MCV,87,87.0,fL,NaN
3,10098672,51200,Eosinophils,0.3,0.3,%,NaN
4,10098672,51250,MCV,83,83.0,fL,NaN


In [163]:
lab_patients = df.subject_id.value_counts().index

寻找哪些 label 是所有人都做过的

In [164]:
tmp = df.groupby(['subject_id', 'label']).agg({
    'subject_id': 'count'
}).index.to_list()

res = pd.Series([_[1] for _ in tmp]).value_counts()

In [165]:
res[res > 2400]

White Blood Cells                2412
Hematocrit                       2412
Red Blood Cells                  2412
RDW                              2412
Platelet Count                   2412
MCV                              2412
MCHC                             2412
MCH                              2412
Hemoglobin                       2412
Glucose                          2409
Creatinine                       2407
Estimated GFR (MDRD equation)    2407
Urea Nitrogen                    2406
Sodium                           2402
Potassium                        2402
Chloride                         2402
Name: count, dtype: int64

In [170]:
for _label in res[res > 2400].index.tolist():
    print('='*100)
    ap = both_ibd_patients + only_cd_patients + only_uc_patiens
    for i ,_mode in enumerate([ap, both_ibd_patients, only_cd_patients, only_uc_patiens]):
        filter_condition = (df.subject_id.isin(_mode)) \
                                & (df.label == _label)
        if i == 0:
            print(df[filter_condition].valueuom.value_counts())
            print('-'*100)
        print('label: {}\nmode: {}'.format(_label, i))
        print(df[filter_condition].valuenum.describe()[1:3])
        print('- '*50)

valueuom
K/uL    370415
Name: count, dtype: int64
----------------------------------------------------------------------------------------------------
label: White Blood Cells
mode: 0
mean    8.615597
std     4.948404
Name: valuenum, dtype: float64
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
label: White Blood Cells
mode: 1
mean    8.780767
std     4.724537
Name: valuenum, dtype: float64
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
label: White Blood Cells
mode: 2
mean    8.546286
std     4.552827
Name: valuenum, dtype: float64
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
label: White Blood Cells
mode: 3
mean    8.688134
std     5.699616
Name: valuenum, dtype: float64
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
valueuom
%    389042
Name: count, dtype: int64
------

In [152]:
data = np.array(df[df['label'] == 'Glucose'].valuenum.dropna())

In [160]:
df[df.valuenum == 1276100]

,subject_id,itemid,label,value,valuenum,valueuom,dod
187731,12468016,50809,Glucose,1276103,1276100.0,mg/dL,NaN
191703,12468016,50813,Lactate,1276103,1276100.0,mmol/L,NaN
230779,12468016,50809,Glucose,1276103,1276100.0,mg/dL,NaN
234751,12468016,50813,Lactate,1276103,1276100.0,mmol/L,NaN
273827,12468016,50809,Glucose,1276103,1276100.0,mg/dL,NaN
...,...,...,...,...,...,...,...
8348540,12468016,50813,Lactate,1276103,1276100.0,mmol/L,NaN
8387616,12468016,50809,Glucose,1276103,1276100.0,mg/dL,NaN
8391588,12468016,50813,Lactate,1276103,1276100.0,mmol/L,NaN
8860537,12468016,50809,Glucose,1276103,1276100.0,mg/dL,NaN
